In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -qU --upgrade pip
!pip install -qU kaggle ultralytics

In [ ]:
!ls drive/MyDrive/cse396_datasets/no_banana_dataset.zip

In [ ]:
# 2️⃣ Unzip the dataset archive into Colab local storage
dataset_zip = '/content/drive/MyDrive/cse396_datasets/no_banana_dataset.zip'
!unzip -q "$dataset_zip" -d /content/no_banana_dataset
!ls /content/no_banana_dataset


In [ ]:
# 3️⃣ Count images & labels per split (corrected paths)
import os

base = '/content/no_banana_dataset/no_banana_dataset'
splits = ['train', 'valid', 'test']

for s in splits:
    img_dir = os.path.join(base, s, 'images')
    lbl_dir = os.path.join(base, s, 'labels')
    n_img = len([f for f in os.listdir(img_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))])
    n_lbl = len([f for f in os.listdir(lbl_dir) if f.endswith('.txt')])
    print(f"{s:5s}: {n_img:5d} images, {n_lbl:5d} labels")


In [ ]:
# 4️⃣ Display a sample image + YOLO-format box (using corrected paths)
import cv2
import matplotlib.pyplot as plt
import os

base = '/content/no_banana_dataset/no_banana_dataset'

def show_sample(split='train', idx=0):
    img_dir = os.path.join(base, split, 'images')
    lbl_dir = os.path.join(base, split, 'labels')
    imgs = sorted([f for f in os.listdir(img_dir) if f.lower().endswith(('.jpg','.png','.jpeg'))])
    img_name = imgs[idx]
    img_path = os.path.join(img_dir, img_name)
    lbl_path = os.path.join(lbl_dir, img_name.rsplit('.',1)[0] + '.txt')

    img = cv2.imread(img_path)
    h, w = img.shape[:2]
    with open(lbl_path) as f:
        for line in f:
            cls, x_c, y_c, bw, bh = map(float, line.split())
            x1 = int((x_c - bw/2) * w)
            y1 = int((y_c - bh/2) * h)
            x2 = int((x_c + bw/2) * w)
            y2 = int((y_c + bh/2) * h)
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    plt.figure(figsize=(6, 6))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')

# Show the first sample in train
show_sample('train', 0)


In [ ]:
%%bash
cat > /content/no_banana_dataset/data.yaml <<EOF
# Dataset config for Ultralytics YOLOv8
path: /content/no_banana_dataset/no_banana_dataset    # root directory for train/ valid/ test/
train: train/images
val:   valid/images
test:  test/images

nc: 6
names: ['apple_fresh', 'apple_rotten',
        'potato_fresh', 'potato_rotten',
        'orange_fresh', 'orange_rotten']
EOF

# verify
echo "---- data.yaml ----"
cat /content/no_banana_dataset/data.yaml


In [ ]:
from ultralytics import YOLO

# Initialize a YOLOv8n model
model = YOLO("yolov8n.pt")

# Create checkpoint directory in Drive
checkpoint_dir = '/content/drive/MyDrive/yolo_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

model.train(
    data="/content/no_banana_dataset/data.yaml",
    epochs=200,
    patience=50,
    imgsz=640,
    batch=16,
    lr0=0.005,
    cos_lr=True,
    mosaic=1.0,
    mixup=0.4,
    freeze=10,
    save_period=20,  # Save checkpoint every 20 epochs
    project="/content/drive/MyDrive/runs_no_banana/train",  # Save to Drive
    name="no_banana"
)

In [ ]:
!ls drive/MyDrive/runs_no_banana/train/no_banana/weights